In [ ]:
import webbrowser
from spotifyconnector.spotify_api import get_spotify_oauth

auth = get_spotify_oauth("credentials.json", "user-modify-playback-state user-read-playback-state user-read-recently-played")
auth_url = auth.get_authorize_url()
webbrowser.open(auth_url)

In [ ]:
from spotipy import Spotify

ACCESS_TOKEN = ""
token_info = auth.get_access_token(ACCESS_TOKEN)
access_token = token_info["access_token"]
spotify = Spotify(auth=access_token)

In [ ]:
import json

recently = spotify.current_user_recently_played(limit=50)
json.dump(recently, open("recent.json", "w"), indent=4)


In [ ]:
summary = {}
for song in recently:
    track_id = song["track"]["id"]
    track_name = song["track"]["name"]
    played_at = song["played_at"]

    analysis = spotify.audio_features(track_id)[0]
    danceability = analysis["danceability"]
    energy = analysis["energy"]
    speechiness = analysis["speechiness"]
    acousticness = analysis["acousticness"]


    summary[track_name] = {
        "track_name": track_name,
        "played_at": played_at,
        "danceability": danceability,
        "energy": energy,
        "speechiness": speechiness,
        "acousticness": acousticness,
    }

json.dump(summary, open("summary.json", "w"), indent=4)

In [ ]:
songs = [song for song, data in summary.items()]
danceability = [data["danceability"] for song, data in summary.items()]
acousticness = [data["acousticness"] for song, data in summary.items()]

In [ ]:
zipped_lists = zip(danceability, acousticness, songs)
differences = [(abs(y1 - y2), x) for y1, y2, x in zipped_lists]
differences.sort(reverse=True)
sorted_songs = [x for _, x in differences]

In [ ]:
sorted_dance = [summary[song]["danceability"] for song in sorted_songs]
sorted_acoustic = [summary[song]["acousticness"] for song in sorted_songs]

In [ ]:
import matplotlib.pyplot as plt

_, ax = plt.subplots()

ax.plot(sorted_songs[0:20], sorted_dance[0:20], label="danceability")
ax.plot(sorted_songs[0:20], sorted_acoustic[0:20], label="acousticness")
ax.legend()
ax.grid()
ax.xaxis.set_tick_params(rotation=45)
